In [2]:
import torch
print(torch.__version__)

2.7.1+cpu


In [ ]:
x = torch.tensor([2.0], requires_grad=True)
y = x**2
y.backward() # 자동으로 dy/dx 미분값 계산
print(x.grad) # y=2x에 x=[2,0]이니까 

tensor([4.])


Python에서 _ (언더스코어)의 의미


약속: "내부용입니다. 건드리지 마세요"
```python
class Tensor:
    def __init__(self, data):
        self.data = data          # 공개: 사용자가 접근 OK
        self._backward = None     # 내부용: 직접 건드리지 말 것
        self._prev = []           # 내부용
```
📦 비유
```python

# 스마트폰
class Phone:
    def call(self, number):       # 공개 기능 (사용자가 씀)
        self._connect_network()   # 내부 기능 (자동으로 실행됨)
    
    def _connect_network(self):   # 내부용 (사용자가 직접 안 씀)
        pass

# 사용자는
phone.call("010-1234-5678")  # ✅ 이렇게만 씀
# phone._connect_network()   # ❌ 이건 직접 안 부름
```

#### 목표 Backward 만들기

In [4]:
class Domino:
    def __init__(self,name):
        self.name = name
        self.next = None #다음도미노 링크드 리스트랑 비슷함
        self._fall_animation = None #각도미노가 쓰러지는 방식

    def fall(self):  # 공개 함수
        # 1. 내 애니메이션 실행
        self._fall_animation()
        
        # 2. 다음 도미노도 쓰러뜨리기
        if self.next:
            self.next.fall()  # 재귀!

In [5]:
# 도미노 3개 만들기
d1 = Domino("첫째")
d2 = Domino("둘째")
d3 = Domino("셋째")

# 연결
d1.next = d2
d2.next = d3

# 각자 다른 쓰러지는 애니메이션 정의
d1._fall_animation = lambda: print("첫째: 천천히 쿵...")
d2._fall_animation = lambda: print("둘째: 빠르게 탁!")
d3._fall_animation = lambda: print("셋째: 크게 쾅!!")

In [6]:
d1.fall()  # 첫 도미노만 건드림


첫째: 천천히 쿵...
둘째: 빠르게 탁!
셋째: 크게 쾅!!


```
d1.fall() 호출
  → d1._fall_animation() 실행 (첫째: 천천히 쿵...)
  → d2.fall() 호출
      → d2._fall_animation() 실행 (둘째: 빠르게 탁!)
      → d3.fall() 호출
          → d3._fall_animation() 실행 (셋째: 크게 쾅!!)
```

In [7]:
class Tensor:
    def __init__(self, data, requires_grad=False):
        self.data = data
        self.grad = None
        self.requires_grad = requires_grad
        self._backward = lambda: None
        self._prev = []
    
    def mul(self, other):
        # Forward (기존 코드 그대로)
        result_data = []
        for i in range(len(self.data)):
            row = []
            for j in range(len(self.data[0])):
                row.append(self.data[i][j] * other.data[i][j])
            result_data.append(row)
        
        out = Tensor(result_data)
        out._prev = [self, other]  # 부모 기록
        
        # Backward 정의
        def _backward():
            # grad 초기화 (None이면)
            if self.grad is None:
                self.grad = [[0.0] * len(self.data[0]) for _ in range(len(self.data))]
            if other.grad is None:
                other.grad = [[0.0] * len(other.data[0]) for _ in range(len(other.data))]
            
            # Chain rule 적용
            for i in range(len(self.data)):
                for j in range(len(self.data[0])):
                    self.grad[i][j] += out.grad[i][j] * other.data[i][j]
                    other.grad[i][j] += out.grad[i][j] * self.data[i][j]
        
        out._backward = _backward
        return out
    
    def backward(self):
        # 시작점: grad = 1로 초기화
        if self.grad is None:
            self.grad = [[1.0] * len(self.data[0]) for _ in range(len(self.data))]
        
        # 내 _backward 실행
        self._backward()
        
        # 부모들에게 전파 (도미노!)
        for parent in self._prev:
            parent.backward()

In [8]:
x = Tensor([[2.0]], requires_grad=True)
y = x.mul(x)  # y = x²

y.backward()
print(x.grad)  # [[4.0]] 나와야 함

[[4.0]]
